In [49]:
import string
#import sys  ## importing necessary libraries
#sys.path.insert(0, "/work/zerostart/.local/lib/python3.6/site-packages")
 ## always have this
import pprint
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim 
import re
import numpy as np
# importing libraries
import datetime
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import readline
todaysdate = datetime.datetime.now().strftime("%Y-%m-%d")
print(todaysdate)
from io import open
import glob
import os
import unicodedata
import string
import gensim.models.word2vec as w2v
import csv
import string
PAD_CHAR = "<PAD>"
UNK_CHAR = "<UNK>"
import operator
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
from collections import defaultdict
from scipy.sparse import csr_matrix


class ProcessedIter(object):

    def __init__(self, Y, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename) as f:
            r = csv.reader(f)
            next(r)
            for row in r:
                yield (row[3].split())

                
def word_embeddings(Y, notes_file, embedding_size, min_count, n_iter):
    modelname = "processed_%s.w2v" % (Y)
    sentences = ProcessedIter(Y, notes_file)

    model = w2v.Word2Vec(size=embedding_size, min_count=min_count, workers=4, iter=n_iter)
    print("building word2vec vocab on %s..." % (notes_file))
    
    model.build_vocab(sentences)
    print("training...")
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    out_file = '/'.join(notes_file.split('/')[:-1] + [modelname])
    print("writing embeddings to %s" % (out_file))
    model.save(out_file)
    return out_file


def findFiles(path):
    """
    returns a list of files in a path
    """
    return glob.glob(path)


def unicodeToAscii(s):
    """
    Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


def normalizeStringAndDigitsLower(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([,.:!?])", r" \1 ", s)
    s = re.sub(r"[\|]+", r" ", s)
    s = re.sub('\n', '<NEWLINE>', s)
    s = re.sub('\t', ' ', s)
    s = re.sub('\d+', '#', s)
    s = re.sub(' +', ' ',s)
    return s


def build_vocab(vocab_min, infile, vocab_filename):
    """
        INPUTS:
            vocab_min: how many documents a word must appear in to be kept
            infile: (training) data file to build vocabulary from
            vocab_filename: name for the file to output
    """
    with open(infile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        #header
        next(reader)

        #0. read in data
        print("reading in data...")
        #holds number of terms in each document
        note_numwords = []
        #indices where notes start
        note_inds = [0]
        #indices of discovered words
        indices = []
        #holds a bunch of ones
        data = []
        #keep track of discovered words
        vocab = {}
        #build lookup table for terms
        num2term = {}
        #preallocate array to hold number of notes each term appears in
        note_occur = np.zeros(400000, dtype=int)
        i = 0
        for row in reader:
            text = row[2]
            numwords = 0
            for term in text.split():
                #put term in vocab if it's not there. else, get the index
                index = vocab.setdefault(term, len(vocab))
                indices.append(index)
                num2term[index] = term
                data.append(1)
                numwords += 1
            #record where the next note starts
            note_inds.append(len(indices))
            indset = set(indices[note_inds[-2]:note_inds[-1]])
            #go thru all the word indices you just added, and add to the note occurrence count for each of them
            for ind in indset:
                note_occur[ind] += 1
            note_numwords.append(numwords)
            i += 1
        #clip trailing zeros
        note_occur = note_occur[note_occur>0]

        #turn vocab into a list so indexing doesn't get fd up when we drop rows
        vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

        #1. create sparse document matrix
        C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
        #also need the numwords array to be a sparse matrix
        note_numwords = csr_matrix(1. / np.array(note_numwords))
        
        #2. remove rows with less than 3 total occurrences
        print("removing rare terms")
        #inds holds indices of rows corresponding to terms that occur in < 3 documents
        inds = np.nonzero(note_occur >= vocab_min)[0]
        print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
        #drop those rows
        C = C[inds,:]
        note_occur = note_occur[inds]
        vocab_list = vocab_list[inds]

        print("writing output")
        with open(vocab_filename, 'w') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")
                

n_letters

def letterToIndex(letter):
    """
    find index for all letters
    """
    return all_letters.find(letter)


def letterToTensor(letter):
    """
    Just for demonstration, turn a letter into a <1 x n_letters> Tensor
    """
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


def lineToTensor(line):
    """# Turn a line into a <line_length x 1 x n_letters>, or an array of one-hot letter vectors"""
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


all_CNER_words = {}
all_CNER_labels = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]


# for filename in findFiles('data/*.txt'):
#     category = os.path.splitext(os.path.basename(filename))[0]
#     all_CNER_labels.append(category)
#     lines = readLines(filename)
#     category_lines[category] = lines

# n_categories = len(all_categories)



def to_scalar(var):
    """
    returns a python float
    """
    return var.view(-1).data.tolist()[0]


def argmax(vec):
    """
     return the argmax as a python int
    """
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.cuda.LongTensor(idxs)
    return autograd.Variable(tensor)


def log_sum_exp(vec):
    """
        function to Compute log sum exp in a numerically stable way for the forward algorithm
    Args:
        vector
    Returns:
        log sum exp
    """
    
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))



def scores_to_tags(scores, tag_to_ix):
    '''
    function to transform scores for one sentence to tags for this sentence
    Args:
        scores: n*d torch variable, n is the number sequence length, d is the tag numbers
        tag_to_ix: dict that converts a tag to a integer distinctly
    Returns:
        tags: tag list of length n
    '''
    scores = scores.data.numpy()
    tags_idxs = [np.max(score) for score in scores]
    tags = [[key for key in tag_to_ix.keys() if tag_to_ix[key] == idx][0] for idx in tags_idxs]
    return tags

# def tags_to_i2b2(idx, sentence, tags):
#     '''
#     Args:
#         idx: line index
#         sentence: a list of words in the sentence
#         tags: a list of tags in 'IOB' format of the sentence
#     Return:
#         i2b2_list: a list of all the entities recognized in i2b2 format
#     '''
#     i2b2_list = []
#     start = -1
#     end = start
#     label = None
#     #tags = scores_to_tags(scores, tag_to_ix)
#     for i in range(len(tags)):
#         if 'B' in tags[i]:
#             start = i
#             end = i
#             label = tags[i][2:]
#         elif 'I' in tags[i]:
#             end = i
#         else:
#             if start > -1 and end >= start:
#                 i2b2_list.append('c="{0}" {1}:{2} {1}:{3}||t="{4}"'.format((' ').join(sentence[start:end+1]), idx, start, end, label))
#             start = -1
#             end = start
#             label = None
#     return i2b2_list


# def predict(model, txt_file, con_file, output_dir):
#     '''
#     Args:
#         model: model saved by torch.save() command
#         txt_file: one .txt file path
#         con_fil: one .con file path
#         output_dir: directory for store the outcome .con file
    
#     '''
#     test_sentences, test_labels = file_to_list(txt_file, con_file)
#     origin_total = 0
#     pred_total = 0
#     intersection_total = 0
#     list_output = []
#     for j in range(len(test_sentences)):
#         sentence = test_sentences[j]
#         label = test_labels[j]
#         print(sentence)
#         print(label)
#         #print(tags_to_i2b2(j, sentence, label))
#         origin = tags_to_i2b2(j, sentence, label)
#         origin_total += len(origin)
        
#         if sentence == []:
#             continue
#         inputs, inputs_char = prepare_sequence_char(sentence, word_to_ix, char_to_ix)
#         tag_scores = model(inputs, inputs_char)
#         print(scores_to_tags(tag_scores, tag_to_ix))
#         #print(tags_to_i2b2(j, sentence, scores_to_tags(tag_scores, tag_to_ix)))
#         pred = tags_to_i2b2(j, sentence, scores_to_tags(tag_scores, tag_to_ix))
#         pred_total += len(pred)
#         if len(pred) > 0:
#             list_output.append('\n'.join(pred))
        
#         print(set(origin) & set(pred))
#         intersection_total += len(set(origin) & set(pred))
#         print("\n",intersection_total)
    
#     basename = os.path.basename(txt_file).split('.')[0]
#     output_path = os.path.join(output_dir, basename + '.con')
#     if os.path.exists(output_path):
#         os.remove(output_path)
#     with open(output_path, 'w') as f:
#         f.write('\n'.join(list_output))
        
#     precision = intersection_total / pred_total
#     recall = intersection_total / origin_total
#     print("precision:{0:.2f}".format(precision))
#     print("recall:{0:.2f}".format(recall))
#     print("f measure:{0:.2f}:".format(2 * precision * recall / (precision + recall)))


def get_ner_fmeasure(golden_lists, predict_lists):
    sent_num = len(golden_lists)
    golden_full = []
    predict_full = []
    right_full = []
    right_tag = 0
    all_tag = 0
    right_num = 0
    golden_num = 0
    predict_num = 0

    for idx in range(0,sent_num):
        # word_list = sentence_lists[idx]
        golden_list = golden_lists[idx]
        predict_list = predict_lists[idx]
        for idy in range(len(golden_list)):
            if golden_list[idy] == predict_list[idy]:
                right_tag += 1
        all_tag += len(golden_list)
        right_ner = list(set(golden_list).intersection(set(predict_list)))
        golden_full += golden_list
        predict_full += predict_list
        right_full += right_ner
    right_num = len(right_full)
    golden_num = len(golden_full)
    predict_num = len(predict_full)
    if predict_num == 0:
        precision = -1
    else:
        precision =  (right_num+0.0)/predict_num
    if golden_num == 0:
        recall = -1
    else:
        recall = (right_num+0.0)/golden_num
    if (precision == -1) or (recall == -1) or (precision+recall) <= 0.:
        f_measure = -1
    else:
        f_measure = 2*precision*recall/(precision+recall)
    accuracy = (right_tag+0.0)/all_tag
    return accuracy, precision, recall, f_measure



# pn ="/work/data/COMP/DDICorpusBrat/Train/MedLine/"
# txt_names = glob.glob(pn + "*.txt")
# drug_bank_names_list = []
# for txt in txt_names:
#     t=txt.split("/")
#     name_want=t[7].split(".")[0]
#     drug_bank_names_list.append(name_want)

    
# unique_drug_bank_names = list(set(drug_bank_names_list))
# print(unique_drug_bank_names)


# zip_files_list = []
# print('the unique number of drug-drug interaction (DDI) file names in this annotated set is', len(unique_drug_bank_names))
# for unique_name in unique_drug_bank_names:
#     txt_file =pn + unique_name + ".txt"
#     ann_file =pn + unique_name + ".ann"
#     files=(txt_file, ann_file)
#     zip_files_list.append(files)

def spacy_file_to_list(txt_file, con_file):
    '''
    Argv:
        txt_filepath:
        con_filepath
    Return:
        sentences:
        labels:
    '''
    #txt_file = glob.glob(txt_filepath)
    #con_file = glob.glob(con_filepath)
    #print(txt_file)
    with open(txt_file) as tf:
        content = tf.read()
        lines = content #.split("\n")
        #sentences = [line.split() for line in lines]
        #labels = [["O" for w in list_word] for list_word in lines] 
    tf.close()

    list_groups = []
    with open(con_file) as cf:
        for line in cf.readlines():
            if not line.strip():
                continue
            concept_regex ="^(T\d+)\t(\w+)\s(\d+)\s(\d+)\t(\w+)"
            match = re.search(concept_regex, line.strip())
            if match:
                groups = match.groups()
                list_groups.append([groups[0],groups[1], int(groups[2]), int(groups[3]), groups[4]])
        #print(list_groups) 
    cf.close()
    list_spacy = []
    labels = {'entities': list_spacy}
    for entity in list_groups:
        #print(entity)
        #idx_line = entity[0]
        start = entity[2]
        end = entity[3]
        label = entity[1]
        list_spacy.append(tuple([start, end, label]))
    return tuple([lines, labels])

files_list=findFiles('/Users/isabelmetzger/PycharmProjects/NaturalLanguageInterface/track2-training_data_1/*.txt')
unique_file_names=[file.split('/')[6].split('.')[0] for file in files_list]
print(unique_file_names)
        
print("letter to tensor examples and line to tensor examples for character levels")
print('p letter to tensor', letterToTensor('p'))

print('Lipitor line to tensor', lineToTensor('Lipitor').size())

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

2018-08-05
['101372', '102283', '102296', '102527', '101566', '101215', '102668', '102456', '102324', '101410', '103317', '100187', '103074', '100579', '100035', '103677', '101276', '102019', '101857', '102027', '102557', '102967', '100590', '101331', '100039', '101092', '100564', '103722', '100229', '103293', '102173', '102629', '102365', '101136', '100677', '102698', '101813', '100847', '102937', '101779']
letter to tensor examples and line to tensor examples for character levels
p letter to tensor tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
Lipitor line to tensor torch.Size([7, 1, 57])


In [50]:
zip_files_list = []
pn = '/Users/isabelmetzger/PycharmProjects/NaturalLanguageInterface/track2-training_data_1/'
print('the unique number of n2c2 track2 training data file names in this annotated set is', len(unique_file_names))
for unique_name in unique_file_names:
    txt_file =pn + unique_name + ".txt"
    ann_file =pn + unique_name + ".ann"
    files=(txt_file, ann_file)
    zip_files_list.append(files)
import pickle
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([spacy_file_to_list(zip_files_list[i][0], zip_files_list[i][1]) for i in range(len(zip_files_list))], f)


the unique number of n2c2 track2 training data file names in this annotated set is 40


In [51]:
with open('objs.pkl', 'rb') as f:
    data = pickle.load(f)
    
# loading pickle
#then printing out some of the entities

for i in range(10):#len(data)):
    pprint.pprint(data[i][1].get('entities'))

[(10009, 10015, 'Form'),
 (10016, 10018, 'Route'),
 (10046, 10055, 'Drug'),
 (10056, 10061, 'Strength'),
 (10062, 10068, 'Form'),
 (10084, 10090, 'Form'),
 (10091, 10093, 'Route'),
 (10112, 10122, 'Drug'),
 (10123, 10129, 'Strength'),
 (10130, 10137, 'Form'),
 (10151, 10158, 'Form'),
 (10159, 10161, 'Route'),
 (10989, 10999, 'Drug'),
 (11001, 11005, 'Strength'),
 (1137, 1148, 'Drug'),
 (1150, 1158, 'Drug'),
 (1159, 1163, 'Strength'),
 (1164, 1166, 'Route'),
 (1235, 1246, 'Drug'),
 (1248, 1251, 'Strength'),
 (1252, 1254, 'Route'),
 (1255, 1261, 'Drug'),
 (1263, 1266, 'Strength'),
 (1267, 1269, 'Route'),
 (1270, 1276, 'Drug'),
 (1417, 1425, 'Drug'),
 (1430, 1438, 'Reason'),
 (1579, 1582, 'Drug'),
 (1583, 1587, 'Strength'),
 (1588, 1590, 'Route'),
 (1591, 1593, 'Frequency'),
 (164, 175, 'Drug'),
 (5995, 6003, 'Drug'),
 (6016, 6026, 'Drug'),
 (6039, 6046, 'Drug'),
 (6047, 6049, 'Dosage'),
 (6059, 6067, 'Drug'),
 (6716, 6725, 'Drug'),
 (6729, 6734, 'Strength'),
 (6735, 6740, 'Frequency'),
 

 (7361, 7372, 'Drug'),
 (7373, 7379, 'Strength'),
 (7380, 7385, 'Form'),
 (7386, 7396, 'Route'),
 (7397, 7404, 'Drug'),
 (7405, 7410, 'Strength'),
 (7411, 7416, 'Frequency'),
 (7417, 7427, 'Drug'),
 (7428, 7434, 'Strength'),
 (7435, 7440, 'Frequency'),
 (7468, 7475, 'Drug'),
 (7476, 7481, 'Strength'),
 (7557, 7559, 'Route'),
 (7560, 7570, 'Frequency'),
 (7575, 7586, 'Drug'),
 (7587, 7592, 'Strength'),
 (7593, 7599, 'Form'),
 (7613, 7619, 'Form'),
 (7620, 7622, 'Route'),
 (7623, 7633, 'Frequency'),
 (7638, 7644, 'Drug'),
 (7645, 7651, 'Strength'),
 (7652, 7659, 'Form'),
 (7673, 7680, 'Form'),
 (7681, 7683, 'Route'),
 (7684, 7695, 'Frequency'),
 (7700, 7706, 'Drug'),
 (7707, 7712, 'Strength'),
 (7713, 7719, 'Form'),
 (7733, 7739, 'Form'),
 (7740, 7742, 'Route'),
 (7778, 7782, 'Reason'),
 (7787, 7796, 'Drug'),
 (7797, 7801, 'Strength'),
 (7802, 7808, 'Form'),
 (7822, 7828, 'Form'),
 (7829, 7831, 'Route'),
 (7832, 7842, 'Frequency'),
 (7847, 7858, 'Drug'),
 (7859, 7863, 'Strength'),
 (7864

 (8942, 8952, 'Drug'),
 (8953, 8958, 'Strength'),
 (8959, 8960, 'Dosage'),
 (8971, 8979, 'Route'),
 (9022, 9032, 'Drug'),
 (9033, 9038, 'Strength'),
 (9039, 9041, 'Route'),
 (9042, 9047, 'Frequency'),
 (9052, 9062, 'Drug'),
 (9063, 9068, 'Strength'),
 (9081, 9089, 'Route'),
 (9090, 9095, 'Frequency'),
 (9127, 9138, 'Drug'),
 (9139, 9144, 'Strength'),
 (9145, 9147, 'Route'),
 (9148, 9151, 'Frequency'),
 (9185, 9196, 'Drug'),
 (9197, 9202, 'Strength'),
 (9203, 9204, 'Dosage'),
 (9215, 9223, 'Route'),
 (9224, 9235, 'Frequency'),
 (9267, 9277, 'Drug'),
 (9278, 9284, 'Strength'),
 (9285, 9287, 'Route'),
 (9288, 9290, 'Frequency'),
 (9295, 9305, 'Drug'),
 (9306, 9312, 'Strength'),
 (9313, 9314, 'Dosage'),
 (9326, 9334, 'Route'),
 (9378, 9389, 'Drug'),
 (9390, 9395, 'Strength'),
 (9396, 9398, 'Route'),
 (9399, 9401, 'Frequency'),
 (9406, 9417, 'Drug'),
 (9418, 9423, 'Strength'),
 (9424, 9425, 'Dosage'),
 (9436, 9444, 'Route'),
 (9488, 9498, 'Drug'),
 (9499, 9503, 'Strength'),
 (9504, 9506, 'R

In [72]:
def creating_stats_new_files(con_file):
    list_groups = []
    with open(con_file) as cf:
        for line in cf.readlines():
            #print(line)
            if not line.strip():
                continue
            concept_regex ="^(T\d+)\t(\w+)\s(\d+)\s(\d+)\t(\w+)"
            match = re.search(concept_regex, line.strip())
            if match:
                groups= match.groups()
                list_groups.append([(groups[0]), groups[1], groups[2], groups[3], groups[4]])
    cf.close()
    for entity in list_groups:
        print(entity)
        character_span = int(entity[3]) - int(entity[2])
#         print(character_span)
        


In [73]:
ann_files=findFiles('/Users/isabelmetzger/PycharmProjects/NaturalLanguageInterface/track2-training_data_1/*.ann')
creating_stats_new_files(ann_files[0])

['T1', 'Reason', '10302', '10331', 'Hypoglycemia']
29
['T2', 'ADE', '10365', '10377', 'Hypoglycemia']
12
['T3', 'Drug', '10430', '10437', 'insulin']
7
['T4', 'Drug', '10514', '10521', 'insulin']
7
['T5', 'Drug', '10580', '10587', 'insulin']
7
['T6', 'Dosage', '10809', '10822', 'sliding']
13
['T7', 'Drug', '10851', '10854', 'NPH']
3
['T8', 'Dosage', '10859', '10872', 'sliding']
13
['T9', 'Reason', '11223', '11254', 'Chronic']
31
['T10', 'Drug', '11325', '11344', 'cardiac']
19
['T11', 'Reason', '11651', '11654', 'CAD']
3
['T12', 'Drug', '11768', '11771', 'ASA']
3
['T13', 'Drug', '11773', '11779', 'plavix']
6
['T14', 'Drug', '11781', '11787', 'statin']
6
['T15', 'Drug', '11789', '11791', 'BB']
2
['T16', 'Reason', '11880', '11899', 'Atrial']
19
['T17', 'Drug', '11917', '11925', 'coumadin']
8
['T18', 'Drug', '11954', '11964', 'metoprolol']
10
['T20', 'Strength', '12407', '12412', '80']
5
['T22', 'Strength', '12425', '12430', '20']
5
['T23', 'Drug', '12432', '12446', 'Ranitidine']
14
['T24',

In [88]:
def remove_punc_completely(s):
    """
    Args:
    input: string (text)
    output: 
    """
    s = re.sub('([,!?:;])', '', s)
    return s


def normalize_with_space_to_punc(s):
    """
    Args:
    input: string (text)
    output:
    """
    s = re.sub('([.,!?:;])', r' \1 ', s)
    s = re.sub('\s{2,}', ' ', s)
    return s


def remove_brackets(s):
    """
    since mimic has lots of brackets
    input:string s
    output s without brackets
    """
    s = re.sub('\[\*\*.*\*\*\]', ' ', s).replace('  ', ' ')
    return s


def replace_digits(s, character):
    """
    
    normalize digits with either "d" or "#" character
    input : string, character is character you would like to replace digit with
    output: new string
    """
    
    s = re.sub("[0-9]", character, s)
    return s


def replace_break(s):
    """replacing repititive breaks
    input: text
    output: txt with new line put in
    
    """
    s = re.sub('\\n', '<NEW_LINE>', s).replace('  ', ' ')
    return s


print("example of a note:\n===========================================\n")
replace_break(replace_digits(remove_brackets(remove_punc_completely(data[0][0])), character="#"))

example of a note:



'Admission Date <NEW_LINE><NEW_LINE>Date of Birth    Sex F<NEW_LINE><NEW_LINE>Service MEDICINE<NEW_LINE><NEW_LINE>Allergies<NEW_LINE>Penicillins<NEW_LINE><NEW_LINE>Attending <NEW_LINE>Chief Complaint<NEW_LINE>Stridor<NEW_LINE><NEW_LINE>Major Surgical or Invasive Procedure<NEW_LINE>mechanical ventilation<NEW_LINE>Extubation<NEW_LINE><NEW_LINE><NEW_LINE>History of Present Illness<NEW_LINE>Ms. is a ##F with schizoaffective disorder<NEW_LINE>polysubstance dependence and SAH s/p coiling L MCA aneurysm<NEW_LINE> who initially presented to OSH ED with sore throat x #<NEW_LINE>week. Neck X-ray was performed and revealed possible thickened<NEW_LINE>epiglottis but she left the ED AMA prior to finalized .<NEW_LINE>She then returned to the OSH ED at approximately ##pm with new<NEW_LINE>onset stridor and hoarseness. She was also noted to be altered<NEW_LINE>with slurred speech and there was concern for intoxication. She<NEW_LINE>was intubated with a #-# ETT for airway protection with emesis<NEW_LIN

In [78]:
data[0][0]

'Admission Date:  [**2130-10-2**]              Discharge Date:   [**2130-10-4**]\n\nDate of Birth:  [**2085-9-4**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nPenicillins\n\nAttending:[**First Name3 (LF) 5810**]\nChief Complaint:\nStridor\n\nMajor Surgical or Invasive Procedure:\nmechanical ventilation\nExtubation\n\n\nHistory of Present Illness:\nMs. [**Known lastname 87445**] is a 45F with schizoaffective disorder,\npolysubstance dependence, and SAH s/p coiling L MCA aneurysm\n[**2130-7-2**] and repeat coiling for recannalization on [**9-20**], [**2129**] who initially presented to OSH ED with sore throat x 1\nweek.  Neck X-ray was performed and revealed possible thickened\nepiglottis but she left the ED AMA prior to finalized [**Location (un) 1131**].\nShe then returned to the OSH ED at approximately 10pm with new\nonset stridor and hoarseness. She was also noted to be altered\nwith slurred speech and there was concern for intoxication. She\nwas intubated with a 7-0 ETT